In [ ]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*4)])
  except RuntimeError as e:
    print(e)

In [ ]:
imsize=(64,64)
ipsize=imsize+(3,)
data=tf.keras.utils.image_dataset_from_directory('dataset/images/',image_size=imsize)
'''data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    # Add more preprocessing layers as needed
])
data = data.map(lambda x, y: (data_augmentation(x, training=True), y))'''

In [ ]:
#Label 1 is DOG
#Label 0 is CAT
train_size=int(len(data)*.8)
val_size=int(len(data)*.2)+1
#test_size=int(len(data)*.1)+1
#all_size=len(data)

In [ ]:
data=data.map(lambda x,y:(x/255,y))

In [ ]:
train=data.take(len(data))
val=data.skip(train_size).take(val_size)
#test=data.skip(train_size+val_size).take(test_size)
#all_data=data.take(all_size)
#train=tf.keras.utils.image_dataset_from_directory('dataset/train/',image_size=imsize,validation_split=0.2,subset="training",seed=42,batch_size=32)
#val=tf.keras.utils.image_dataset_from_directory('dataset/train/',image_size=imsize,validation_split=0.2,subset="validation",seed=42,batch_size=32)
#train=train.map(lambda x,y:(x/255,y))
#val=val.map(lambda x,y:(x/255,y))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,BatchNormalization,Dropout,GlobalAveragePooling2D

In [ ]:
model=Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=ipsize))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())


model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(64, activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.25))  # Optional dropout layer to reduce overfitting
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam' ,loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
logdir='logs'
tensor_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist=model.fit(train,validation_data=val,epochs=20,callbacks=[tensor_callback])

In [ ]:
fig=plt.figure()
plt.plot(hist.history['loss'],color='teal',label='loss')
plt.plot(hist.history['val_loss'],color='orange',label='val_loss')
fig.suptitle('Loss',fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
fig=plt.figure()
plt.plot(hist.history['accuracy'],color='teal',label='accuracy')
plt.plot(hist.history['val_accuracy'],color='orange',label='val_accuracy')
fig.suptitle('Accuracy',fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
model.save(os.path.join('models','catdogclassifier.h5'))

In [ ]:
'''from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy
pre=Precision()
re=Recall()
acc=BinaryAccuracy()'''

In [ ]:
'''for batch in test.as_numpy_iterator():
    x,y=batch
    yhat=model.predict(x)
    pre.update_state(y,yhat)
    re.update_state(y,yhat)
    acc.update_state(y,yhat)
print('Precision:',pre.result().numpy(),'Recall:',re.result().numpy(),'Accuracy:',acc.result().numpy())'''